# 03 — Model Eğitimi (Training)

Bu notebook `src.train` modülünün mantığını adım adım gösterir:
- Cross-validation ile 4 modeli karşılaştırır (baseline, RF, XGB, LGB)
- CV skoru istatistiklerini görselleştirir
- HPO (hiperparametre optimizasyonu) nasıl entegre edilir — kısa demo
- Eğitilen modeli `models/` dizinine kaydeder

In [ ]:
import sys

sys.path.insert(0, "..")

import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, StratifiedKFold

plt.rcParams["figure.figsize"] = (10, 4)
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False

## 1. Veri Hazırlama

In [ ]:
from src.io import read_input_dataset
from src.config import Paths
from src.preprocess import preprocess_basic
from src.split import split_dataset

paths = Paths()
df, _ = read_input_dataset(paths.raw_data)
df = preprocess_basic(
    df, target_col="is_canceled", label_map={"no": 0, "yes": 1, 0: 0, 1: 1}
)
split = split_dataset(df, target_col="is_canceled")
X_train, y_train = split.X_train, split.y_train
X_test, y_test = split.X_test, split.y_test
print(f"Train: {X_train.shape}, Test: {X_test.shape}")

## 2. Model Karşılaştırması (5-fold Stratified CV)

In [ ]:
from src.features import build_pipeline
from src.hpo import get_candidate_models

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = {}

for model_name, model in get_candidate_models().items():
    pipe = build_pipeline(model=model)
    scores = cross_val_score(
        pipe, X_train, y_train, cv=cv, scoring="roc_auc", n_jobs=-1
    )
    results[model_name] = scores
    print(f"{model_name:20s} AUC = {scores.mean():.4f} ± {scores.std():.4f}")

In [ ]:
# Görselleştirme: boxplot
fig, ax = plt.subplots()
data = list(results.values())
labels = list(results.keys())
bp = ax.boxplot(data, patch_artist=True, labels=labels)
colors = ["#3b82f6", "#10b981", "#f59e0b", "#ef4444"]
for patch, c in zip(bp["boxes"], colors[: len(data)]):
    patch.set_facecolor(c)
    patch.set_alpha(0.7)
ax.set_title("5-Fold Stratified CV — ROC AUC")
ax.set_ylabel("ROC AUC")
ax.axhline(0.8, color="grey", linestyle="--", lw=1, label="0.80 referans")
ax.legend()
plt.tight_layout()
plt.show()

## 3. En İyi Modeli Eğit ve Kaydet

In [ ]:
from src.hpo import get_candidate_models

# En yüksek ortalama AUC'u olan modeli seç
best_model_name = max(results, key=lambda k: results[k].mean())
print(f"En iyi model: {best_model_name}")

best_model = get_candidate_models()[best_model_name]
best_pipe = build_pipeline(model=best_model)
best_pipe.fit(X_train, y_train)
print("Model eğitildi.")

In [ ]:
# CLI'daki cmd_train ile resmi kayıt için:
# $ python main.py train
#
# Notebook'tan kayıt için:
import joblib
import json
import time
from pathlib import Path

run_id = time.strftime("%Y%m%d_%H%M%S")
out_dir = Path("..") / "models" / run_id
out_dir.mkdir(parents=True, exist_ok=True)

model_path = out_dir / "model.joblib"
joblib.dump(best_pipe, model_path)
print(f"Model kaydedildi: {model_path}")

meta = {
    "run_id": run_id,
    "model": best_model_name,
    "cv_auc_mean": float(results[best_model_name].mean()),
    "cv_auc_std": float(results[best_model_name].std()),
}
(out_dir / "metadata.json").write_text(json.dumps(meta, indent=2))
print(json.dumps(meta, indent=2))

## Sonuç

| Adım | Araç |
|------|------|
| CV değerlendirme | `StratifiedKFold(n_splits=5)` + `roc_auc` |
| HPO | `src.hpo.run_hpo` (Optuna, 50 deneme) |
| Kayıt | `models/<run_id>/model.joblib` + `metadata.json` |
| Resmi kayıt | `python main.py train` |

Bir sonraki notebook: `04_evaluation.ipynb`